In [174]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Dense
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import roc_curve, accuracy_score, mean_squared_error,recall_score,roc_auc_score,r2_score,mean_squared_error
from keras.optimizers import adagrad
from keras.layers import Activation
from sklearn.externals import joblib


In [19]:
df_processed_data = pd.read_csv('processed/imputed_dataset_task2.csv')

In [34]:
df_processed_data.head(10)

,date_time,y,co_gt,nhmc,c6h6,s2,nox,s3,no2,s4,s5,t,rh,ah,level_High,level_Low,level_Moderate,level_Very High,level_Very low,y_later
0,2004-03-10 18:00:00,1360,2.600000,150.0,11.9,1024.412919,166.000000,1056.0,113.0,1692.000000,1077.382582,13.600000,48.900000,0.757800,1,0,0,0,0,1185.0
1,2004-03-10 19:00:00,1292,2.000000,112.0,9.4,955.000000,103.000000,1174.0,92.0,1559.000000,972.000000,13.300000,47.700000,0.725500,1,0,0,0,0,1136.0
2,2004-03-10 20:00:00,1402,2.200000,88.0,9.0,939.000000,131.000000,1140.0,114.0,1555.000000,1074.000000,11.900000,71.699035,0.750200,1,0,0,0,0,1094.0
3,2004-03-10 21:00:00,1376,2.200000,80.0,9.2,948.000000,172.000000,1092.0,122.0,1584.000000,1203.000000,11.000000,60.000000,0.786700,1,0,0,0,0,1010.0
4,2004-03-10 22:00:00,1272,1.600000,51.0,6.5,836.000000,131.000000,1205.0,116.0,1409.984952,1110.000000,31.449763,59.600000,0.788800,1,0,0,0,0,1011.0
5,2004-03-10 23:00:00,1197,1.200000,38.0,4.7,750.000000,89.000000,1337.0,96.0,1393.000000,949.000000,11.200000,59.200000,0.784800,1,0,0,0,0,1066.0
6,2004-03-11 00:00:00,1185,1.200000,31.0,3.6,690.000000,62.000000,1462.0,77.0,1333.000000,733.000000,11.300000,56.800000,0.760300,1,0,0,0,0,1052.0
7,2004-03-11 01:00:00,1136,-3.136557,31.0,3.3,672.000000,12.909905,1453.0,76.0,1333.000000,730.000000,10.700000,60.000000,20.885781,0,0,1,0,0,1144.0
8,2004-03-11 02:00:00,1094,0.900000,24.0,2.3,609.000000,45.000000,1579.0,60.0,1276.000000,620.000000,10.700000,59.700000,0.764800,0,1,0,0,0,1333.0
9,2004-03-11 03:00:00,1010,0.600000,19.0,1.7,561.000000,-200.000000,1705.0,-200.0,1235.000000,501.000000,10.300000,60.200000,0.751700,0,1,0,0,0,1351.0


In [96]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(df_processed_data.drop(['date_time','y','y_later'],axis=1), df_processed_data[['y_later']], test_size=0.15, random_state=9)
# Train validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=9)

In [167]:
def keras_model(activation_function = Activation('relu')):
    model = Sequential()
    model.add(Dense(40, input_dim=X_train.shape[1]))
    model.add(activation_function)
    model.add(Dense(30))
    model.add(activation_function)
    model.add(Dense(10))
    model.add(activation_function)
    model.add(Dense(20))
    model.add(activation_function)
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=adagrad())
    return model

In [170]:
estimator = KerasRegressor(build_fn=keras_model, epochs=10, batch_size=10, verbose=1)
estimator.fit(X_train, y_train.values.reshape(-1,1))


Epoch 1/10
6079/6079 [==============================] - 1s 196us/step - loss: 76997.6369
Epoch 2/10
6079/6079 [==============================] - 1s 114us/step - loss: 68853.0310
Epoch 3/10
6079/6079 [==============================] - 1s 115us/step - loss: 68001.9078
Epoch 4/10
6079/6079 [==============================] - 1s 114us/step - loss: 67506.6470
Epoch 5/10
6079/6079 [==============================] - 1s 115us/step - loss: 66943.2699
Epoch 6/10
6079/6079 [==============================] - 1s 115us/step - loss: 66742.7378
Epoch 7/10
6079/6079 [==============================] - 1s 115us/step - loss: 66672.9481
Epoch 8/10
6079/6079 [==============================] - 1s 115us/step - loss: 66429.5191
Epoch 9/10
6079/6079 [==============================] - 1s 115us/step - loss: 66216.0697
Epoch 10/10
6079/6079 [==============================] - 1s 114us/step - loss: 66119.4844


In [171]:
print("Train R2 score is:", r2_score(y_train, estimator.predict(X_train)).round(3))
print("Train RMSE is: {} \n".format(np.sqrt(mean_squared_error(y_train, estimator.predict(X_train)).round(3))))
   
print("Validation R2 score is:", r2_score(y_val, estimator.predict(X_val)).round(3))
print("Validation RMSE is {} \n".format(np.sqrt(mean_squared_error(y_val, estimator.predict(X_val))).round(3)))
    
print("Test R2 score is:", r2_score(y_test,  estimator.predict(X_test)).round(3))
print("Test RMSE: {} ... \n".format(np.sqrt(mean_squared_error(y_test, estimator.predict(X_test)).round(3))))

6079/6079 [==============================] - 0s 82us/step
Train R2 score is: 0.424
6079/6079 [==============================] - 0s 55us/step
Train RMSE is: 256.1147321026262 

1073/1073 [==============================] - 0s 59us/step
Validation R2 score is: 0.407
1073/1073 [==============================] - 0s 60us/step
Validation RMSE is 264.156 

1263/1263 [==============================] - 0s 61us/step
Test R2 score is: 0.447
1263/1263 [==============================] - 0s 61us/step
Test RMSE: 261.0114537716688 ... 

